In [ ]:
# Этот код проверяет кластеризацию вокруг скилов Кловери 


import pandas as pd
import requests
import json
import time
import random
from tqdm import tqdm

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""  
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 20
REQUEST_DELAY = 3

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты эксперт по IT-навыкам и кластеризации данных. Проверь корректность сопоставления IT-навыков:
1. Канон (canon_original) — это эталонный навык
2. Синоним (synonym_original) — предлагаемое соответствие

Для каждой пары выполни два действия:
1. Проверка: является ли canon_original IT-навыком (IT-сфера: программирование, инфраструктура, данные, кибербезопасность и т.д.)
2. Проверка: является ли synonym_original корректным синонимом canon_original

Ответ строго в формате JSON:
{
  "results": [
    {"it_skill": true/false, "correct_cluster": true/false},
    ...
  ]
}
"""

def prepare_prompt(batch):
    pairs = "\n".join([f"{i+1}. Канон: {row['canon_original']} | Синоним: {row['synonym_original']}" 
                      for i, row in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Проверь следующие пары:\n{pairs}\n\nОтвет строго в формате JSON с ключом 'results':"}
    ]

def query_mistral(prompt, max_tokens=900):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()
            
            # Удаляем возможные обёртки ```json
            if content.startswith("```json"):
                content = content[7:-3].strip()
            
            # Попытка "почистить" JSON
            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")
            
            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def process_excel(file_path, output_path):
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    matched_sheets = {name: df for name, df in all_sheets.items() if name.endswith('_matched')}
    results = {}

    for sheet_name, df in matched_sheets.items():
        print(f"\nОбработка листа: {sheet_name} ({len(df)} строк)")

        df['IT'] = None
        df['Mistral_check'] = None

        for i in tqdm(range(0, len(df), BATCH_SIZE)):
            batch = df.iloc[i:i+BATCH_SIZE].to_dict('records')
            prompt = prepare_prompt(batch)
            response = query_mistral(prompt)

            if response and 'results' in response:
                for j, item in enumerate(response['results'][:len(batch)]):
                    idx = df.index[i + j]
                    df.at[idx, 'IT'] = item.get('it_skill', False)
                    df.at[idx, 'Mistral_check'] = item.get('correct_cluster', False)
            else:
                print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

            df.to_excel(output_path, index=False)

            if i + BATCH_SIZE < len(df):
                time.sleep(REQUEST_DELAY + random.random())

        results[sheet_name] = df

    with pd.ExcelWriter(output_path) as writer:
        for sheet_name, df in results.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    return results

# Запуск
if __name__ == "__main__":
    input_file = "data/final_Up_skills_mapping_with_originals_no_new_cluster.xlsx"
    output_file = "data/verified_skills.xlsx"
    verified_data = process_excel(input_file, output_file)
    print(f"\n✅ Проверка завершена! Результаты сохранены в: {output_file}")



Обработка листа: ru_matched (6102 строк)


100%|██████████| 306/306 [43:49<00:00,  8.59s/it]



Обработка листа: en_matched (2218 строк)


100%|██████████| 111/111 [16:46<00:00,  9.06s/it]



Обработка листа: ru_en_matched (11837 строк)


100%|██████████| 592/592 [1:31:07<00:00,  9.24s/it]



✅ Проверка завершена! Результаты сохранены в: data/verified_skills.xlsx


In [7]:
df_ru_matched = pd.read_excel('data/verified_skills.xlsx', sheet_name='ru_matched')
df_en_matched = pd.read_excel('data/verified_skills.xlsx', sheet_name='en_matched')
df_ru_en_matched = pd.read_excel('data/verified_skills.xlsx', sheet_name='ru_en_matched')

In [8]:
df_ru_matched

,language,canon_original,synonym_original,IT,Mistral_check
0,ru,Дизайн-мышление,Алгоритмическое мышление,False,False
1,ru,Дизайн-мышление,коммерческое мышление,False,False
2,ru,Дизайн-мышление,Гибкость мышления,False,False
3,ru,Дизайн-мышление,Стратегическое мышление,False,False
4,ru,Дизайн-мышление,- Аналитическое мышление;,False,False
...,...,...,...,...,...
6097,ru,Методы экспертных оценок,Метод Центра оценки,True,False
6098,ru,Установка и настройка системного ПО,Опыт установки и настройки пользовательского ПО,True,False
6099,ru,Анализ процессов и их эффективности,Анализ проблем и их решение,True,False
6100,ru,Негативное тестирование,Негативное тестирование,True,True


In [23]:
match1 = df_ru_matched[df_ru_matched['Mistral_check'] == True]['canon_original'].nunique()
print(match1)

630


In [24]:
match2 = df_en_matched[df_en_matched['Mistral_check'] == True]['canon_original'].nunique()
print(match2)

308


In [25]:
match3 = df_ru_en_matched[df_ru_en_matched['Mistral_check'] == True]['canon_original'].nunique()
print(match3)

130


In [27]:
total = match1 + match2 + match3
print(total)

1068


In [ ]:
# Это код проверяет превичную кластьеризацию раздельных навыков по ru, en, ru_en


import pandas as pd
import requests
import json
import time
import random
from tqdm import tqdm

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""  
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 25
REQUEST_DELAY = 3

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты эксперт по IT-навыкам и кластеризации данных. Проверь корректность сопоставления IT-навыков:
1. Канон (proposed_canon) — это эталонный навык
2. Синоним (skill_original) — предлагаемое соответствие

Для каждой пары выполни два действия:
1. Проверка: является ли proposed_canon IT-навыком (IT-сфера: программирование, инфраструктура, данные, кибербезопасность и т.д.)
2. Проверка: является ли skill_original корректным синонимом proposed_canon

Ответ строго в формате JSON:
{
  "results": [
    {"it_skill": true/false, "correct_cluster": true/false},
    ...
  ]
}
"""

def prepare_prompt(batch):
    pairs = "\n".join([f"{i+1}. Канон: {row['proposed_canon']} | Синоним: {row['skill_original']}" 
                      for i, row in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Проверь следующие пары:\n{pairs}\n\nОтвет строго в формате JSON с ключом 'results':"}
    ]

def query_mistral(prompt, max_tokens=950):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()
            
            # Удаляем возможные обёртки ```json
            if content.startswith("```json"):
                content = content[7:-3].strip()
            
            # Попытка "почистить" JSON
            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")
            
            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def process_excel(file_path, output_path):
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    matched_sheets = {name: df for name, df in all_sheets.items() if name.endswith('_clusters')}
    results = {}

    for sheet_name, df in matched_sheets.items():
        print(f"\nОбработка листа: {sheet_name} ({len(df)} строк)")

        df['IT'] = None
        df['Mistral_check'] = None

        for i in tqdm(range(0, len(df), BATCH_SIZE)):
            batch = df.iloc[i:i+BATCH_SIZE].to_dict('records')
            prompt = prepare_prompt(batch)
            response = query_mistral(prompt)

            if response and 'results' in response:
                for j, item in enumerate(response['results'][:len(batch)]):
                    idx = df.index[i + j]
                    df.at[idx, 'IT'] = item.get('it_skill', False)
                    df.at[idx, 'Mistral_check'] = item.get('correct_cluster', False)
            else:
                print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

            df.to_excel(output_path, index=False)

            if i + BATCH_SIZE < len(df):
                time.sleep(REQUEST_DELAY + random.random())

        results[sheet_name] = df

    with pd.ExcelWriter(output_path) as writer:
        for sheet_name, df in results.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    return results

# Запуск
if __name__ == "__main__":
    input_file = "data/final_skills_mapping_Up_only_clasters.xlsx"
    output_file = "data/verified_clusters.xlsx"
    verified_data = process_excel(input_file, output_file)
    print(f"\n✅ Проверка завершена! Результаты сохранены в: {output_file}")



Обработка листа: ru_clusters (1405 строк)


100%|██████████| 57/57 [09:43<00:00, 10.24s/it]



Обработка листа: en_clusters (2654 строк)


100%|██████████| 107/107 [18:35<00:00, 10.43s/it]



Обработка листа: ru_en_clusters (2538 строк)


100%|██████████| 102/102 [16:06<00:00,  9.48s/it]



✅ Проверка завершена! Результаты сохранены в: data/verified_clusters.xlsx


In [ ]:
# Это код проверяет вториную  кластеризацию навыков HDBSCAN без шума


import pandas as pd
import requests
import json
import time
import random
from tqdm import tqdm



API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""  
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 25
REQUEST_DELAY = 3

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты эксперт по IT-навыкам и кластеризации данных. Проверь корректность сопоставления IT-навыков:
1. Канон (canon) — это эталонный навык
2. Синоним (synonym) — предлагаемое соответствие

Для каждой пары выполни два действия:
1. Проверка: является ли canon IT-навыком (IT-сфера: программирование, инфраструктура, данные, кибербезопасность и т.д.)
2. Проверка: является ли synonym корректным синонимом canon

Ответ строго в формате JSON:
{
  "results": [
    {"it_skill": true/false, "correct_cluster": true/false},
    ...
  ]
}
"""

def prepare_prompt(batch):
    pairs = "\n".join([f"{i+1}. Канон: {row['canon']} | Синоним: {row['synonym']}" 
                      for i, row in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Проверь следующие пары:\n{pairs}\n\nОтвет строго в формате JSON с ключом 'results':"}
    ]

def query_mistral(prompt, max_tokens=950):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()
            
            # Удаляем возможные обёртки ```json
            if content.startswith("```json"):
                content = content[7:-3].strip()
            
            # Попытка "почистить" JSON
            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")
            
            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def process_excel(file_path, output_path):
    # Читаем только первый лист 
    df = pd.read_excel(file_path, sheet_name=0)  # sheet_name=0 - первый лист
    
    print(f"\nОбработка листа: {len(df)} строк)")

    df['IT'] = None
    df['Mistral_check'] = None

    for i in tqdm(range(0, len(df), BATCH_SIZE)):
        batch = df.iloc[i:i+BATCH_SIZE].to_dict('records')
        prompt = prepare_prompt(batch)
        response = query_mistral(prompt)

        if response and 'results' in response:
            for j, item in enumerate(response['results'][:len(batch)]):
                idx = df.index[i + j]
                df.at[idx, 'IT'] = item.get('it_skill', False)
                df.at[idx, 'Mistral_check'] = item.get('correct_cluster', False)
        else:
            print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

        # Сохраняем после каждого батча 
        df.to_excel(output_path, index=False)

        if i + BATCH_SIZE < len(df):
            time.sleep(REQUEST_DELAY + random.random())

    # Сохраняем финальный результат
    df.to_excel(output_path, index=False)
    return df

# Запуск
if __name__ == "__main__":
    input_file = "data/canon_synonmyms_mc3_ms1.xlsx"
    output_file = "data/verified_canon_synonmyms_mc3_ms1.xlsx"
    verified_data = process_excel(input_file, output_file)
    print(f"\n✅ Проверка завершена! Результаты сохранены в: {output_file}")


Обработка листа: 4500 строк)


  6%|▌         | 11/180 [06:44<1:29:23, 31.74s/it]


Ошибка (попытка 1): 429 Client Error: Too Many Requests for url: https://api.mistral.ai/v1/chat/completions
Сырой ответ:
 {"object":"error","message":"Service tier capacity exceeded for this model.","type":"invalid_request_error","param":null,"code":null}


  7%|▋         | 13/180 [07:39<1:20:32, 28.94s/it]


Ошибка (попытка 1): 429 Client Error: Too Many Requests for url: https://api.mistral.ai/v1/chat/completions
Сырой ответ:
 {"object":"error","message":"Service tier capacity exceeded for this model.","type":"invalid_request_error","param":null,"code":null}


...


Ошибка (попытка 1): 429 Client Error: Too Many Requests for url: https://api.mistral.ai/v1/chat/completions
Сырой ответ:
 {"object":"error","message":"Service tier capacity exceeded for this model.","type":"invalid_request_error","param":null,"code":null}


 72%|███████▏  | 129/180 [1:14:03<35:41, 41.99s/it]


Ошибка (попытка 1): 429 Client Error: Too Many Requests for url: https://api.mistral.ai/v1/chat/completions
Сырой ответ:
 {"object":"error","message":"Service tier capacity exceeded for this model.","type":"invalid_request_error","param":null,"code":null}


100%|██████████| 180/180 [1:35:09<00:00, 31.72s/it]



✅ Проверка завершена! Результаты сохранены в: data/verified_canon_synonmyms_mc3_ms1.xlsx


In [ ]:
# exp

import pandas as pd
import requests
import json
import time
import random
from tqdm import tqdm

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 15
REQUEST_DELAY = 5

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты помощник по извлечению опыта работы из вакансий. Для каждого текста определи, сколько у кандидата должен быть опыт работы, и заполни поле `experience` одним из следующих значений:

- "Нет опыта"
- "1–3 года"
- "3–6 лет"
- "Более 6 лет"

Если информация об опыте не указана, поставь null.

Ответ строго в формате JSON:
{
  "results": [
    {"experience": "Нет опыта" / "1–3 года" / "3–6 лет" / "Более 6 лет" / null},
    ...
  ]
}
"""

def prepare_prompt(batch):
    entries = "\n".join([f"{i+1}. {text}" for i, text in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Вот тексты вакансий:\n{entries}\n\nОтвет строго в JSON по формату выше."}
    ]

def query_mistral(prompt, max_tokens=1024):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()

            if content.startswith("```json"):
                content = content[7:-3].strip()

            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")

            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def extract_experience(input_file):
    df = pd.read_csv(input_file)
    if 'experience' not in df.columns:
        df['experience'] = None

    texts = df['text'].fillna("").astype(str).tolist()

    for i in tqdm(range(0, len(df), BATCH_SIZE)):
        batch_texts = texts[i:i+BATCH_SIZE]
        prompt = prepare_prompt(batch_texts)
        response = query_mistral(prompt)

        if response and 'results' in response:
            for j, item in enumerate(response['results'][:len(batch_texts)]):
                idx = i + j
                df.at[idx, 'experience'] = item.get('experience', None)
        else:
            print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

        df.to_csv(input_file, index=False)

        if i + BATCH_SIZE < len(df):
            time.sleep(REQUEST_DELAY + random.random())

    df.to_csv(input_file, index=False)
    print(f"\n✅ Извлечение завершено. Результаты сохранены в: {input_file}")
    return df

# Запуск
if __name__ == "__main__":
    extract_experience("df_to_ml_experience.csv")


  0%|          | 0/13 [00:00<?, ?it/s]


Ошибка (попытка 1): HTTPSConnectionPool(host='api.mistral.ai', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000240E7FD7F10>: Failed to resolve 'api.mistral.ai' ([Errno 11001] getaddrinfo failed)"))


C:\Users\Андрей\AppData\Local\Temp\ipykernel_9116\349819032.py:99: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1–3 года' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'experience'] = item.get('experience', None)
 23%|██▎       | 3/13 [01:07<03:46, 22.62s/it]


Ошибка (попытка 1): ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))


100%|██████████| 13/13 [03:32<00:00, 16.32s/it]


✅ Извлечение завершено. Результаты сохранены в: df_to_ml_experience.csv


In [ ]:
# country 

import pandas as pd
import requests
import json
import time
import random
from tqdm import tqdm

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 30
REQUEST_DELAY = 4

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты географический помощник. По каждому названию города определи страну, в которой он находится.

Ответ строго в формате JSON:
{
  "results": [
    {"country": "Россия"},
    ...
  ]
}

Если город не удаётся распознать или страна неизвестна — верни null.
"""

def prepare_prompt(batch):
    cities = "\n".join([f"{i+1}. {city}" for i, city in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Определи страну для следующих городов:\n{cities}\n\nОтвет строго в JSON по формату выше."}
    ]

def query_mistral(prompt, max_tokens=400):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()

            if content.startswith("```json"):
                content = content[7:-3].strip()

            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")

            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def extract_country(input_file):
    df = pd.read_csv(input_file)
    if 'country' not in df.columns:
        df['country'] = None

    # Берём только строки с непустыми city и пустыми country
    mask = df['city'].notna() & (df['city'].astype(str).str.strip() != "") & df['country'].isna()
    target_df = df[mask].copy()

    cities = target_df['city'].astype(str).tolist()
    index_list = target_df.index.tolist()

    for i in tqdm(range(0, len(cities), BATCH_SIZE)):
        batch_cities = cities[i:i+BATCH_SIZE]
        batch_indices = index_list[i:i+BATCH_SIZE]
        prompt = prepare_prompt(batch_cities)
        response = query_mistral(prompt)

        if response and 'results' in response:
            for j, item in enumerate(response['results'][:len(batch_cities)]):
                idx = batch_indices[j]
                df.at[idx, 'country'] = item.get('country', None)
        else:
            print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

        df.to_csv(input_file, index=False)

        if i + BATCH_SIZE < len(cities):
            time.sleep(REQUEST_DELAY + random.random())

    df.to_csv(input_file, index=False)
    print(f"\n✅ Определение стран завершено. Результаты сохранены в: {input_file}")
    return df

# Запуск
if __name__ == "__main__":
    extract_country("city_country_3.csv")


  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\Андрей\AppData\Local\Temp\ipykernel_15980\3364430923.py:98: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Россия' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'country'] = item.get('country', None)
100%|██████████| 2/2 [00:08<00:00,  4.40s/it]


✅ Определение стран завершено. Результаты сохранены в: city_country_2.csv


In [ ]:
# contry 2 с защитой 

import requests
import json
import pandas as pd
import time
import random
from tqdm import tqdm

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 20
REQUEST_DELAY = 4
INPUT_FILE = "df_city_country.csv"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты географический помощник. По каждому названию города определи страну, в которой он находится.

Ответ строго в формате JSON:
{
  "results": [
    {"country": "Россия"},
    ...
  ]
}

Если город не удаётся распознать или страна неизвестна — верни null.
"""

def prepare_prompt(batch):
    cities = "\n".join([f"{i+1}. {city}" for i, city in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Определи страну для следующих городов:\n{cities}\n\nОтвет строго в JSON по формату выше."}
    ]

def query_mistral(prompt, max_tokens=500):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()

            if content.startswith("```json"):
                content = content[7:-3].strip()

            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")

            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def extract_country(input_file):
    print("🔄 Загрузка текущего состояния файла...")
    df = pd.read_csv(input_file)
    if 'country' not in df.columns:
        df['country'] = None

    # Целевые строки: city непустой и country ещё не указан
    mask = df['city'].notna() & (df['city'].astype(str).str.strip() != "") & df['country'].isna()
    target_df = df[mask].copy()

    cities = target_df['city'].astype(str).tolist()
    index_list = target_df.index.tolist()

    print(f"🔍 Найдено {len(cities)} городов без страны. Начинаем обработку...")

    for i in tqdm(range(0, len(cities), BATCH_SIZE)):
        # Загрузка последней версии файла (на случай падения)
        df = pd.read_csv(input_file)

        batch_cities = cities[i:i + BATCH_SIZE]
        batch_indices = index_list[i:i + BATCH_SIZE]

        # Пропустить батч, если все страны уже определены
        already_done = all(pd.notna(df.loc[idx, 'country']) for idx in batch_indices)
        if already_done:
            continue

        prompt = prepare_prompt(batch_cities)
        response = query_mistral(prompt)

        if response and 'results' in response:
            for j, item in enumerate(response['results'][:len(batch_cities)]):
                idx = batch_indices[j]
                df.at[idx, 'country'] = item.get('country', None)
        else:
            print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

        df.to_csv(input_file, index=False)

        if i + BATCH_SIZE < len(cities):
            time.sleep(REQUEST_DELAY + random.random())

    print(f"\n✅ Обработка завершена. Файл сохранён: {input_file}")
    return df

# Запуск
if __name__ == "__main__":
    extract_country(INPUT_FILE)


🔄 Загрузка текущего состояния файла...
🔍 Найдено 118 городов без страны. Начинаем обработку...


 83%|████████▎ | 5/6 [00:44<00:09,  9.10s/it]


Ошибка (попытка 1): HTTPSConnectionPool(host='api.mistral.ai', port=443): Read timed out. (read timeout=120)


100%|██████████| 6/6 [03:06<00:00, 31.11s/it]


✅ Обработка завершена. Файл сохранён: df_city_country.csv
